# **Notebook for Working with "Krainiy Sever" Newspaper (Murgin Nuteneut Section)**

This notebook builds the following datasets:
1. Parallel dataset (Chukchi and Russian sentences)
2. Monolingual dataset (Chukchi texts)

Preliminary text file processing is required:
- Chukchi text should come first, followed by Russian
- Separate Chukchi and Russian texts using "ru:"
- Visually inspect for name/date correlations
- Work with dictionary to identify untranslated segments marked with "\*\*\*Chukchi-text/Russian-text\*\*\*"
- Mark clearly parallel segments using "&&&Chukchi-text&&&" and "\$\$\$Russian-text\$\$\$"

If a title lacks Russian translation, mark it as "БЕЗ ЗАГОЛОВКА" (if title exists only in Russian - remove it)

In [ ]:
from http.cookiejar import unmatched
!pip install pandas
! pip install nltk

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   -------- ------------------------------- 2.4/11.5 MB 14.9 MB/s eta 0:00:01
   --------------------- ------------------ 6.3/11.5 MB 21.5 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 21.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   --------- ------------------------------ 3.1/12.6 MB 18.5 MB/s eta 0:00:01
   ----------------------------- ---------- 9.2/12.6 MB 21.2 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 19.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import csv
import pandas as pd

import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize

import shutil

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\eliza\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eliza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = pd.DataFrame(columns=['ckt', 'ru', 'filename'])
df.to_csv("parallel_dataset_krainiy_sever_addition.csv", index=False)

In [ ]:
df.head()

,ckt,ru,filename


In [ ]:
# Folder paths
SOURCE_FOLDER = 'C:\\Users\\eliza\\PycharmProjects\\course_project\\chukot_articles\\remake'
DONE_FOLDER = 'C:\\Users\\eliza\\PycharmProjects\\course_project\\chukot_articles\\done'
MONO_FOLDER = 'C:\\Users\\eliza\\PycharmProjects\\course_project\\chukot_articles\\mono_and_strange'
DATASET_PATH = 'parallel_dataset_krainiy_sever_addition.csv'

PARTIALLY_DONE_FOLDER = "C:\\Users\\eliza\\PycharmProjects\\course_project\\chukot_articles\\partially_done"

In [ ]:
# Creating DataFrame if file doesn't exist
# if not os.path.exists(DATASET_PATH):
#     df = pd.DataFrame(columns=['ckt', 'ru'])
#     df.to_csv(DATASET_PATH, index=False)

Opening file:

In [ ]:
def open_file(filename: str) -> str:
    file_path = os.path.join(SOURCE_FOLDER, filename)
    with open(file_path, 'r', encoding='utf-8') as txt:
        return txt.read()

Splitting texts into Chukchi and Russian:

In [ ]:
def ckt_and_russian(file_text: str):
    if 'ru:' in file_text:
        return file_text.split('ru:')
    return None, None

Preprocessing (removing empty lines and ***-enclosed lines):

In [ ]:
def preprocess(text: str):
    filtered_passage = [
        passage for passage in text.split('\n')
        if not (passage.startswith(('***', '&&&', '$$$')) and passage.rstrip().endswith(('***', '&&&', '$$$')))
    ]
    return [passage.strip() for passage in filtered_passage if passage.strip()]

Splitting paragraphs into sentences:

In [ ]:
def sentences_tokenizer(text_list: list[str]):
    tokenized_text = []
    for passage in text_list:
        tokenized_text.append(sent_tokenize(passage))
    return tokenized_text

Processing single file (adding sentence pairs to dataset):

In [ ]:
def process_file(filename):
    """Processes single file, adding suitable sentence pairs to dataset"""
    file_text = open_file(filename)

    ckt_text, ru_text = ckt_and_russian(file_text)

    # Verifying successful language separation
    if ckt_text is None or ru_text is None:
        print(f"Processing error (no language separation): {filename}")
        return False

    # Text preprocessing
    ckt_text_preprocessed = preprocess(ckt_text)
    ru_text_preprocessed = preprocess(ru_text)

    # Checking paragraph count
    if len(ckt_text_preprocessed) != len(ru_text_preprocessed):
        print(f"Processing error (paragraph count mismatch): {filename}")
        return False

    # Sentence tokenization by paragraphs
    ckt_sentences = sentences_tokenizer(ckt_text_preprocessed)
    ru_sentences = sentences_tokenizer(ru_text_preprocessed)

    # Collecting matching paragraphs
    matched_ckt, matched_ru = [], []
    unmatched_paragraphs = []

    # Verifying sentence count matches in each paragraph
    for i, (ckt_paragraph, ru_paragraph) in enumerate(zip(ckt_sentences, ru_sentences)):
        if len(ckt_paragraph) == len(ru_paragraph): # if sentence counts match
            matched_ckt.extend(ckt_paragraph)
            matched_ru.extend(ru_paragraph)
        else:
            unmatched_paragraphs.append(i + 1)  # Saving paragraph numbers that didn't match


    # Adding matching sentences to dataset if any exist
    if matched_ckt:
        new_data = pd.DataFrame({
            'ckt': matched_ckt,
            'ru': matched_ru,
            'filename': filename
            #'status': 'partial' if unmatched_paragraphs else 'full'
        })

    df = pd.read_csv(DATASET_PATH)
    df = pd.concat([df, new_data], ignore_index=True)
    df.to_csv(DATASET_PATH, index=False)

    # Moving processed file
    if unmatched_paragraphs:
        print(f"Partially processed file: {filename}, unprocessed paragraphs: {unmatched_paragraphs}")
        shutil.move(os.path.join(SOURCE_FOLDER, filename), os.path.join(PARTIALLY_DONE_FOLDER, filename))
    else:
        print(f"Fully processed file: {filename}")
        shutil.move(os.path.join(SOURCE_FOLDER, filename), os.path.join(DONE_FOLDER, filename))

    return True

Processing all files in 'remake' folder:

In [ ]:
for file in os.listdir(SOURCE_FOLDER):
    if file.endswith(".txt"):
        process_file(file)

Файл полностью обработан: chukot_article_14391.txt
Файл полностью обработан: chukot_article_9721.txt


Checking resulting dataset:

In [ ]:
df = pd.read_csv("parallel_dataset_krainiy_sever_addition.csv")

In [ ]:
df

,ckt,ru,filename
0,Миӈкыри нынвинэвӄин ӄораӈы,Приручить оленя,chukot_article_14391.txt
1,Гэкэӈыльин ръэлягыргын эр-гаво нинэлгыӄинэт му...,Одной из визитных карточек Чукотки являются го...,chukot_article_14391.txt
2,Ръэ-нут нынтэнмавӄэн Анадырыкэн районак – «Ръи...,"Те, что проводятся в Анадырском районе – «Ръил...",chukot_article_14391.txt
3,Талянъяпкэнайпы ӄорагынрэтыльа ныяаӄэн йыӄыгыр...,Между тем на протяжении веков тундровики испол...,chukot_article_14391.txt
4,"Миӈкыри вальылӄылтэ гакаӈӄор ынкъам мооӄор, ми...","О том, какими данными должны обладать будущие ...",chukot_article_14391.txt
5,– Ёнрыма рывинэвылӄылти нинэги-тэӄинэт ӄрым ыт...,– При выборе будущих ездовых оленей учитываютс...,chukot_article_14391.txt
6,Ӄырым ымыльо ёнрыёт пэнвэлти таӈъяаӈ вальыт ръ...,"Не каждый из бычков, отобранных в упряжь, подх...",chukot_article_14391.txt
7,- нивӄин Сергей Петрович.,– заметил Сергей Петрович.,chukot_article_14391.txt
8,"– Ынанъыттъыёч торрывэнавъёттэ гытлильыт, кыту...",– Первый «отборочный» тур в весеннюю корализац...,chukot_article_14391.txt
9,Нинэгитэӄинэт ымы миӈкыри ныпэраӄэн ӄораӈы – э...,Также внимание обращают на окрас – животные с ...,chukot_article_14391.txt


Function for finding files containing "NO TITLE"

In [ ]:
def bez_zagolovka(filename):
    file_text = open_file(filename)

    for line in file_text.split('\n'):
        if line.rstrip().lstrip().startswith(('БЕЗ ЗАГОЛОВКА', 'БЕЗ НАЗВАНИЯ')):
            print(filename)

In [ ]:
for file in os.listdir(SOURCE_FOLDER):
    if file.endswith(".txt"):
        bez_zagolovka(file)

## **Creating Monolingual Dataset:**

In [ ]:
import os
import pandas as pd
import re

In [ ]:
df_mono = pd.DataFrame(columns=['ckt', 'filename'])
df_mono.to_csv("mono_dataset_krainiy_sever.csv", index=False)

In [ ]:
df_mono.head()

,ckt,filename


In [ ]:
SOURCE_FOLDER_MONO = 'C:\\Users\\eliza\\PycharmProjects\\course_project\\chukot_articles\\done'
DATASET_PATH_MONO = 'mono_dataset_krainiy_sever.csv'

In [ ]:
def open_file_mono(filename: str) -> str:
    file_path = os.path.join(SOURCE_FOLDER_MONO, filename)
    with open(file_path, 'r', encoding='utf-8') as txt:
        return txt.read()

In [ ]:
def sentences_tokenizer_mono(sentences_list: list[str]) -> list[str]:
    sentences = []

    pattern = r'''
        (?<=[.!?])       # Splitting after period, exclamation or question mark
        \s+              # One or more whitespace characters
        (?=[А-ЯA-Z])     # Next character is uppercase (new sentence start)
        |
        (?<=\”)\s*–\s*   # Splitting by dash after closing quote
        |
        \s*–\s*(?=\”)    # Splitting by dash before opening quote
        |
        (?<!\w)\s*–\s*(?!\w)  # Splitting by dash when indicating direct speech
    '''

    for sentence in sentences_list:
        split_sentences = re.split(pattern, sentence, flags=re.VERBOSE)
        # split_sentences = re.split(r'(?<=[.!?])\s+(?=[А-ЯA-Z])', sentence)
        sentences.extend(split_sentences)
    return sentences

In [ ]:
def creating_mono_dataset(filename):
    file_text = open_file_mono(filename)

    ckt_text = []

    for line in file_text.split('\n'):
        if line.startswith('&&&') and line.rstrip().endswith('&&&'):
            ckt_text.append(line.strip('&&&'))

    if ckt_text:
        ckt_sentences = sentences_tokenizer_mono(ckt_text)

        new_data = pd.DataFrame({
            'ckt': ckt_sentences,
            'filename': filename
        })

        df_mono = pd.read_csv(DATASET_PATH_MONO)
        df_mono = pd.concat([df_mono, new_data], ignore_index=True)
        df_mono.to_csv(DATASET_PATH_MONO, index=False)

    return True

In [ ]:
for file in os.listdir(SOURCE_FOLDER_MONO):
    if file.endswith(".txt"):
        creating_mono_dataset(file)

In [ ]:
df_mono = pd.read_csv("mono_dataset_krainiy_sever.csv")
df_mono

,ckt,filename
0,"Мури, нэнэк’эймури, к’ол нинмури нылк’ынмури ы...",chukot_article_10326.txt
1,Уйн’э гыт экэнтикэ?,chukot_article_10326.txt
2,Мытрэкэнтун’ыркын!,chukot_article_10326.txt
3,"Ытлён ынанковчемгъольо гатвален, мури ыныкы ны...",chukot_article_10326.txt
4,Ыннэн н’эвыск’этк’эй Увэлек нымытвальын нылгин...,chukot_article_10326.txt
...,...,...
352,"Чукоткак, гымыкгэта, нэмыӄэй моольылӄылморэ ры...",chukot_article_9721.txt
353,Чинит моргынан ынӄэн мигчир гыму лынъёлӄыл.,chukot_article_9721.txt
354,Лымӈэ тэӈинмэтыркын тэйкык Нутэкэликжэл (слова...,chukot_article_9721.txt
355,Ӄыльугыткы ӄутти нынныкалеёттэ нымнымыт: Кэӈие...,chukot_article_9721.txt


## **Appending Additional Monolingual Data (Chukchi-only Texts without Markers):**

In [ ]:
SOURCE_FOLDER_APPEND = 'C:\\Users\\eliza\\PycharmProjects\\course_project\\chukot_articles\\ne_poluchilos_ne_fortanulo\\stranger_things\\mono'

In [ ]:
def open_file_append(filename: str) -> str:
    file_path = os.path.join(SOURCE_FOLDER_APPEND, filename)
    with open(file_path, 'r', encoding='utf-8') as txt:
        return txt.read()

In [ ]:
def appending_mono_dataset(filename):
    file_text = open_file_append(filename)

    text = file_text.split('\n')
    text = [line for line in text if line.strip()]

    ckt_sentences = sentences_tokenizer_mono(text)

    new_data = pd.DataFrame({
        'ckt': ckt_sentences,
        'filename': filename
    })

    df_mono = pd.read_csv(DATASET_PATH_MONO)
    df_mono = pd.concat([df_mono, new_data], ignore_index=True)
    df_mono.to_csv(DATASET_PATH_MONO, index=False)

    return True

In [ ]:
for file in os.listdir(SOURCE_FOLDER_APPEND):
    if file.endswith(".txt"):
        appending_mono_dataset(file)

In [ ]:
df_mono = pd.read_csv("mono_dataset_krainiy_sever.csv")
df_mono

,ckt,filename
0,"Мури, нэнэк’эймури, к’ол нинмури нылк’ынмури ы...",chukot_article_10326.txt
1,Уйн’э гыт экэнтикэ?,chukot_article_10326.txt
2,Мытрэкэнтун’ыркын!,chukot_article_10326.txt
3,"Ытлён ынанковчемгъольо гатвален, мури ыныкы ны...",chukot_article_10326.txt
4,Ыннэн н’эвыск’этк’эй Увэлек нымытвальын нылгин...,chukot_article_10326.txt
...,...,...
699,Нытъэтӄин ровтынвык,chukot_article_9402_smth_strange.txt
700,"Ыпыкэлиӈэвӄэй,",chukot_article_9402_smth_strange.txt
701,"Мэйӈылылеӈэвӄэй,",chukot_article_9402_smth_strange.txt
702,Ымпытъочгынлгыӈойӈын,chukot_article_9402_smth_strange.txt


## **Creating Another Monolingual Dataset**

In [ ]:
import os
import pandas as pd

In [ ]:
df_mono_extra = pd.DataFrame(columns=['ckt', 'filename'])
df_mono_extra.to_csv("extra_mono_dataset_krainiy_sever.csv", index=False)

In [ ]:
df_mono_extra.head()

,ckt,filename


In [ ]:
SOURCE_FOLDER_MONO_EXTRA = 'C:\\Users\\eliza\\PycharmProjects\\course_project\\chukot_articles\\ne_poluchilos_ne_fortanulo\\stranger_things\\model_for_sopostavlenie\\mono'
DATASET_PATH_MONO_EXTRA = 'extra_mono_dataset_krainiy_sever.csv'

In [ ]:
def open_file_append(filename: str) -> str:
    file_path = os.path.join(SOURCE_FOLDER_MONO_EXTRA, filename)
    with open(file_path, 'r', encoding='utf-8') as txt:
        return txt.read()

In [ ]:
def extra_mono_dataset(filename):
    file_text = open_file_append(filename)

    text = file_text.split('\n')
    text = [line for line in text if line.strip()]

    ckt_sentences = sentences_tokenizer_mono(text)

    new_data = pd.DataFrame({
        'ckt': ckt_sentences,
        'filename': filename
    })

    df_mono_extra = pd.read_csv(DATASET_PATH_MONO_EXTRA)
    df_mono_extra = pd.concat([df_mono_extra, new_data], ignore_index=True)
    df_mono_extra.to_csv(DATASET_PATH_MONO_EXTRA, index=False)

    return True

In [ ]:
for file in os.listdir(SOURCE_FOLDER_MONO_EXTRA):
    if file.endswith(".txt"):
        extra_mono_dataset(file)

In [ ]:
df_mono = pd.read_csv("extra_mono_dataset_krainiy_sever.csv")
df_mono

,ckt,filename
0,ӃЭТЧАНРОТ,chukot_article_11334.txt
1,Гантэнмавлен Эвын ЭКУКЭКИНЭ,chukot_article_11334.txt
2,Ӈутингивик мргынан мыткрычмыркын Эналватгыргын...,chukot_article_11334.txt
3,"Мэлвэчьым титэ ӈан солдаттэ,",chukot_article_11334.txt
4,"Пэлятыльыт маравмайынлянвык,",chukot_article_11334.txt
...,...,...
618,NaN,chukot_article_6923_poetry.txt
619,Обнажит под солнышком грудь:,chukot_article_6923_poetry.txt
620,"Хватит всем молока и счастья,",chukot_article_6923_poetry.txt
621,Хватит матери,chukot_article_6923_poetry.txt


## **Creating Alignment Model Dataset (Containing Approximate Translations)**

In [ ]:
import os
import pandas as pd
import re
import shutil

In [ ]:
df_somnitelno = pd.DataFrame(columns=['ckt', 'ru', 'filename'])
df_somnitelno.to_csv("somnitelno_dataset_krainiy_sever.csv", index=False)

In [ ]:
df_somnitelno.head()

,ckt,ru,filename


In [ ]:
SOURCE_FOLDER_SOMNITELNO = 'C:\\Users\\eliza\\PycharmProjects\\course_project\\chukot_articles\\done'
DATASET_PATH_SOMNITELNO = 'somnitelno_dataset_krainiy_sever.csv'

In [ ]:
def somnitelno_open_file(filename: str) -> str:
    file_path = os.path.join(SOURCE_FOLDER_SOMNITELNO, filename)
    with open(file_path, 'r', encoding='utf-8') as txt:
        return txt.read()

In [ ]:
def somnitelno_creating_dataset(filename):
    file_text = somnitelno_open_file(filename)

    ckt_text = []
    ru_text = []

    for line in file_text.split('\n'):
        if line.startswith('&&&') and line.rstrip().endswith('&&&'):
            ckt_text.append(line.strip('&&&'))
        if line.startswith('$$$') and line.rstrip().endswith('$$$'):
            ru_text.append(line.strip('$$$'))

    ckt_text = ' '.join(ckt_text)
    ru_text = ' '.join(ru_text)

    if ckt_text and ru_text:
        new_data = pd.DataFrame({
            'ckt': [ckt_text],
            'ru': [ru_text],
            'filename': [filename]
        })

        df_somnitelno = pd.read_csv(DATASET_PATH_SOMNITELNO)
        df_somnitelno = pd.concat([df_somnitelno, new_data], ignore_index=True)
        df_somnitelno.to_csv(DATASET_PATH_SOMNITELNO, index=False)

    return True

In [ ]:
for file in os.listdir(SOURCE_FOLDER_SOMNITELNO):
    if file.endswith(".txt"):
        somnitelno_creating_dataset(file)

In [ ]:
df_somnitelno = pd.read_csv("somnitelno_dataset_krainiy_sever.csv")
df_somnitelno

,ckt,ru,filename
0,"Мури, нэнэк’эймури, к’ол нинмури нылк’ынмури ы...","Мы, детишки, знали, что у неё всё есть, иногда...",chukot_article_10326.txt
1,ЫНКЫ НЫНЫЙГУЛЕВӃИНЭТ КЭЛИНЫГЙИВЭТЫЛЬЫЛӃЫЛТИ Ын...,"Это условие я выполнил и был принят на учёбу, ...",chukot_article_10549.txt
2,"А´РЭӇАУТ, ЧАМА КАЛЬАНТО Ынӄэн гатвален ынӈытэӄ...",Однако с её стороны я никогда не замечала отли...,chukot_article_10714.txt
3,Ляйвыма ынан гэкэлилинэт вагыргыт лейвыкин.,"Во время плавания он вёл судовой журнал, ставш...",chukot_article_10934.txt
4,Ӄутти нутэтумгыт гъармиятваленат ынӄэн пограни...,Армейские годы многих ребят из разных республи...,chukot_article_11446.txt
5,Ынӄэнат чимгъуу лыӈык ынкъам винрэтэ Владимир ...,"Первым произведением, принёсшим успех, была по...",chukot_article_11676.txt
6,«Ӄэюуӄэй» лыгъоравэтльэн поэтэн Михаил Ваалгыр...,"«На оленьей упряжке я еду в Алькатваам, меня в...",chukot_article_12021.txt
7,Ынӄъом килвэйыплыткук и’лгытрэтыплыткук анъолы...,После «Кильвэя» обитатели стойбища делились на...,chukot_article_12749.txt
8,Чьэчеӈулвэвкы нытаначгаймаӈӈыӄэнат ынкъам ныта...,"Подобно осеннему празднику вылгыӄаанмат, во вр...",chukot_article_13639.txt
9,Эчги рыгрыг янра нынъалыӈоӄэн нынпылӄэвӄэн вээ...,Примерно через неделю шерсть от шкуры начинала...,chukot_article_13976.txt


## **Compiling Dataset:**

In [ ]:
import pandas as pd

In [ ]:
df_1 = pd.read_csv('parallel_dataset_krainiy_sever.csv')
df_1

,ckt,ru,filename
0,"ВАГЫРГЫТ, Э`МИТЛЁНАТ А`ӃАНТЫЯАТЫӇ ВАЛЬЫТ","ИСТОРИИ, КОТОРЫЕ НЕЛЬЗЯ ЗАБЫТЬ",chukot_article_10325.txt
1,Ӈээкык чавчывэн Вера Грачёва гъурэтлин ванвык ...,Дочь потомственного оленевода Вера Грачёва род...,chukot_article_10325.txt
2,Калеткомготагнэты ӈээккэӄэй гэмэйӈэтлин йытотк...,Там же провела детство с родителями.,chukot_article_10325.txt
3,Гэтэӈыйгулетлин ынан чычеткин лыгъоравэтльаеле...,"Она прекрасно знает родной язык, культуру и тр...",chukot_article_10325.txt
4,Гэкэлиткулин пединститутык.,Она окончила педагогический институт.,chukot_article_10325.txt
...,...,...,...
3404,"ТААРОҢГЫРГЫН, ЭЛЁЛЬЫЧЕТГЫРГЫН ЫНКЪАМ ҢЭЭ’ВИК","ОБРЯД, СОСТЯЗАНИЯ И СВАТОВСТВО",different_files_2.txt
3405,"Вера ТЭВЛЯНТО, урэмигчирэтыльын қорагынрэтгырг...","Вера ТЕВЛЯНТО, ветеран оленеводства, городской...",different_files_2.txt
3406,Талпыӈӈок февральининик март эмнуӈыльыт нытэнм...,В конце февраля – начале марта по всей тундре ...,different_files_2.txt
3407,Лымӈэ,Кстати,different_files_2.txt


In [ ]:
df_2 = pd.read_csv('parallel_dataset_krainiy_sever_addition.csv')
df_2

,ckt,ru,filename
0,Миӈкыри нынвинэвӄин ӄораӈы,Приручить оленя,chukot_article_14391.txt
1,Гэкэӈыльин ръэлягыргын эр-гаво нинэлгыӄинэт му...,Одной из визитных карточек Чукотки являются го...,chukot_article_14391.txt
2,Ръэ-нут нынтэнмавӄэн Анадырыкэн районак – «Ръи...,"Те, что проводятся в Анадырском районе – «Ръил...",chukot_article_14391.txt
3,Талянъяпкэнайпы ӄорагынрэтыльа ныяаӄэн йыӄыгыр...,Между тем на протяжении веков тундровики испол...,chukot_article_14391.txt
4,"Миӈкыри вальылӄылтэ гакаӈӄор ынкъам мооӄор, ми...","О том, какими данными должны обладать будущие ...",chukot_article_14391.txt
5,– Ёнрыма рывинэвылӄылти нинэги-тэӄинэт ӄрым ыт...,– При выборе будущих ездовых оленей учитываютс...,chukot_article_14391.txt
6,Ӄырым ымыльо ёнрыёт пэнвэлти таӈъяаӈ вальыт ръ...,"Не каждый из бычков, отобранных в упряжь, подх...",chukot_article_14391.txt
7,- нивӄин Сергей Петрович.,– заметил Сергей Петрович.,chukot_article_14391.txt
8,"– Ынанъыттъыёч торрывэнавъёттэ гытлильыт, кыту...",– Первый «отборочный» тур в весеннюю корализац...,chukot_article_14391.txt
9,Нинэгитэӄинэт ымы миӈкыри ныпэраӄэн ӄораӈы – э...,Также внимание обращают на окрас – животные с ...,chukot_article_14391.txt


In [ ]:
df_full = pd.concat([df_1, df_2], ignore_index=True)

In [ ]:
df_full

,ckt,ru,filename
0,"ВАГЫРГЫТ, Э`МИТЛЁНАТ А`ӃАНТЫЯАТЫӇ ВАЛЬЫТ","ИСТОРИИ, КОТОРЫЕ НЕЛЬЗЯ ЗАБЫТЬ",chukot_article_10325.txt
1,Ӈээкык чавчывэн Вера Грачёва гъурэтлин ванвык ...,Дочь потомственного оленевода Вера Грачёва род...,chukot_article_10325.txt
2,Калеткомготагнэты ӈээккэӄэй гэмэйӈэтлин йытотк...,Там же провела детство с родителями.,chukot_article_10325.txt
3,Гэтэӈыйгулетлин ынан чычеткин лыгъоравэтльаеле...,"Она прекрасно знает родной язык, культуру и тр...",chukot_article_10325.txt
4,Гэкэлиткулин пединститутык.,Она окончила педагогический институт.,chukot_article_10325.txt
...,...,...,...
3456,"Вэчьым, ынӄэн ипэ миӈкыри рытык, иӈӄун ӈэнъянв...","Возможно, в нашем округе надо искать другой пу...",chukot_article_9721.txt
3457,Ынӄэн мигчир рывъёльвык (Анадырык вама Ымнотае...,Возможны и другие практические шаги в этом нап...,chukot_article_9721.txt
3458,Лымӈэ,Кстати,chukot_article_9721.txt
3459,«КС» приложенияк «Мургин нутэнут» икутъэр гиви...,«КС» в приложении «Мургин нутэнут» в течение н...,chukot_article_9721.txt
